In [2]:
%load_ext autoreload
%autoreload 2
    
%load_ext tensorboard
import sys, os
sys.path.append('../paraphrase/')
from paraphraser_args import ModelArguments, DataTrainingArguments, TrainingArguments
from paraphraser_dataloader import load_dataset_pseudo, load_dataset_pseudo2, load_dataset_pseudo_binary, load_dataset_pseudo_binary_single
from paraphraser_trainer import ParaphraserTrainer
from transformers import AutoTokenizer, AutoModelWithLMHead, HfArgumentParser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
data_dir = "../data/pseudo"
# task = "formality"
# task = "abstract+shakespeare+wiki"
task = "shakespeare"
model_name = "t5-small"
# model_nick = "t5_transfer_shakespeare"
model_nick = "t5_transfer_shakespeare_binary"
model_nick = "t5_transfer_formality_3"
meta_task_type = "binary_single"
meta_task = 'transfer'


output_dir = "../models/"
epochs = "5"
train_batch_size = "16"
eval_batch_size = "16"
save_log_steps = "800"

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    model_name,
    "--model_nick",
    model_nick,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, model_nick),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    save_log_steps,
    "--save_steps",
    save_log_steps,
    "--data_parallel",
    "True",
    "--meta_task",
    meta_task,
    "--meta_task_type",
    meta_task_type
])


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
model = AutoModelWithLMHead.from_pretrained(model_args.model_name_or_path)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /home/vivek/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
    

In [6]:
if training_args.meta_task_type=='intra':
    train_dataset = load_dataset_pseudo(data_args.data_dir, tokenizer, mode="train", tasks = task.split('+'), n_proc=2048)
    dev_dataset = load_dataset_pseudo(data_args.data_dir, tokenizer, mode="dev",  tasks = task.split('+'), n_proc=2048)
elif training_args.meta_task_type=='intra2':
    train_dataset = load_dataset_pseudo2(data_args.data_dir, tokenizer, mode="train", tasks = task.split('+'), n_proc=2048)
    dev_dataset = load_dataset_pseudo2(data_args.data_dir, tokenizer, mode="dev",  tasks = task.split('+'), n_proc=2048)
elif training_args.meta_task_type=='diff':
    train_dataset = load_dataset_pseudo_diff(data_args.data_dir, tokenizer, mode="train", tasks = task.split('+'), n_proc=2048)
    dev_dataset = load_dataset_pseudo_diff(data_args.data_dir, tokenizer, mode="dev",  tasks = task.split('+'), n_proc=2048)
elif training_args.meta_task_type=='joint':
    train_dataset = load_dataset_pseudo_joint(data_args.data_dir, tokenizer, mode="train", tasks = task.split('+'), n_proc=2048)
    dev_dataset = load_dataset_pseudo_joint(data_args.data_dir, tokenizer, mode="dev",  tasks = task.split('+'), n_proc=2048)
elif training_args.meta_task_type=='binary':
    train_dataset = load_dataset_pseudo_binary(data_args.data_dir, tokenizer, mode="train", tasks = task.split('+'), n_proc=2048)
    dev_dataset = load_dataset_pseudo_binary(data_args.data_dir, tokenizer, mode="dev",  tasks = task.split('+'), n_proc=2048)

elif training_args.meta_task_type=='binary_single':
    train_dataset = load_dataset_pseudo_binary_single(data_args.data_dir, tokenizer, mode="train", tasks = task.split('+'), n_proc=2048)
    dev_dataset = load_dataset_pseudo_binary_single(data_args.data_dir, tokenizer, mode="dev",  tasks = task.split('+'), n_proc=2048)


100%|██████████| 5/5 [00:01<00:00,  3.62it/s]


In [7]:
idx = 2000
tokenizer.decode(train_dataset[idx][0], skip_special_tokens=True), tokenizer.decode(train_dataset[idx][2], skip_special_tokens=True)

("transfer: he's going to talk to him.", 'It would be spoke to.')

In [6]:
trainer = ParaphraserTrainer([training_args,model_args, data_args], model, tokenizer, train_dataset, dev_dataset)
trainer.train()

***** Running training *****
Num examples = 97779
Num Epochs = 5.0
Total train batch size = 32
Gradient Accumulation steps = 1
Total optimization steps = 15280.0
Logging steps = 800
Save steps = 800
Iteration:   0%|          | 0/3056 [00:00<?, ?it/s]/home/vivek/anaconda3/envs/marvin/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

Epoch:   0%|          | 0/5 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [8]:
training_args.meta_task

'transfer'

In [4]:
# Eval
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
model = AutoModelWithLMHead.from_pretrained(os.path.join(output_dir, model_nick))

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /home/vivek/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
    

In [11]:
example = "transfer: do you really think so? | input: high | output: low"
# example = "transfer: haha, u really think so? | input to output: high"
# example = "transfer: did you hear about his problems, lol | input formality: low | output formality: high"
# example = "transfer: what're you up to? | target: shakespeare"
t = tokenizer(example, return_tensors='pt')

In [12]:
gen = model.generate(input_ids= t.input_ids, attention_mask = t.attention_mask, num_return_sequences = 3, num_beams = 12)

In [13]:
tokenizer.batch_decode(gen, skip_special_tokens=True)

['do you really think so?',
 'do u really think so?',
 "don't you really think so?"]